1- **API EXERCISE:** 


Browse the moviedb API https://developers.themoviedb.org/3/getting-started/introduction and find the top 5 (5 most average_vote) trending movies for the last week (iterate through all the pages).

For each movie, create a dictionary with name, release date and average vote, put in a list and show it.

Store it in a Collection called Movies in the ADS MongoDB cloud Database

In [1]:
import requests
import json
from tqdm import tqdm

with open('api_key', 'r') as f:
    API_KEY = f.read()

url_trending = 'https://api.themoviedb.org/3/trending/movie/week'
parameters_trending = {"api_key":API_KEY}
fieldnames = ["title", "release_date", "vote_average"]


In [2]:
res = requests.get(url_trending,params=parameters_trending).json()
movies = []

for i in tqdm(range(1, res['total_pages']+1)):
    url_trending = 'https://api.themoviedb.org/3/trending/movie/week?api_key={}&page={}'.format(API_KEY,i)
    res = requests.get(url_trending).json()
    for movie in res["results"]:
        row = {}
        for key in fieldnames:
            if key in movie:
                row[key] = movie[key]
        movies.append(row)

100%|███████████████████████████████████████| 1000/1000 [03:11<00:00,  5.22it/s]


In [3]:
top_5 = sorted(movies, key= lambda x: x['vote_average'], reverse=True)[:5]

In [4]:
top_5

[{'title': 'My Name Is Pauli Murray',
  'release_date': '2021-09-17',
  'vote_average': 10.0},
 {'title': 'My Struggle', 'release_date': '2021-09-24', 'vote_average': 10.0},
 {'title': 'Paradox Lost', 'release_date': '2020-12-05', 'vote_average': 10.0},
 {'title': 'Under the Volcano',
  'release_date': '2021-03-20',
  'vote_average': 10.0},
 {'title': 'Miracle: Letters to the President',
  'release_date': '2021-09-15',
  'vote_average': 10.0}]

In [5]:
import pymongo
import certifi
try:
    #create a credentials.txt file in this folder:
    #first line: Database username (not account username)
    #second line: Database username password (not account password)
    #third line: Databse url with port (you can find it in the cloud dashboard)
    #fourth lin: Database name
    if 'conn' in globals():
        conn.close()
        print("Closing")
    
    with open("credentials.txt", 'r') as f:
        [name,password,url,dbname]=f.read().splitlines()
    conn=pymongo.MongoClient("mongodb+srv://{}:{}@{}".format(name,password,url),tlsCAFile=certifi.where())
    
    print ("Connected successfully!!!")
    
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
conn
db = conn[dbname]
db

Connected successfully!!!


Database(MongoClient(host=['dataagile-shard-00-00.p4jpe.mongodb.net:27017', 'dataagile-shard-00-02.p4jpe.mongodb.net:27017', 'dataagile-shard-00-01.p4jpe.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-ksq9m0-shard-0', ssl=True, ssl_ca_certs='/home/johnny/anaconda3/lib/python3.8/site-packages/certifi/cacert.pem'), 'DataAgile')

In [6]:
conn.test
db = conn['DataAgile']
print (db)
conn.list_database_names()

Database(MongoClient(host=['dataagile-shard-00-00.p4jpe.mongodb.net:27017', 'dataagile-shard-00-02.p4jpe.mongodb.net:27017', 'dataagile-shard-00-01.p4jpe.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-ksq9m0-shard-0', ssl=True, ssl_ca_certs='/home/johnny/anaconda3/lib/python3.8/site-packages/certifi/cacert.pem'), 'DataAgile')


['DataAgile', 'admin', 'local']

In [7]:
db.drop_collection("movies")
collection = db.movies
db.list_collection_names()

[]

In [8]:
x = collection.insert_many(top_5)

In [9]:
conn.close()